In [ ]:
import pandas as pd
import pickle
import numpy as np
import os

from google.colab import files
import io
import time

In [ ]:
# mounting google drive  -> choose files from folder on the left vertical taskbar
from google.colab import drive
drive.mount('/content/drive')

# specify base directory for files
baseDir = '/content/drive/My Drive/ETHZ/AML/Project_1/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##  reading data

In [ ]:
D_X_df = pd.read_csv(baseDir + 'X_train.csv', index_col = 'id')
D_y_df = pd.read_csv(baseDir + 'y_train.csv', index_col = 'id')
D_test_df = pd.read_csv(baseDir + 'X_test.csv', index_col = 'id')

D_X_df.head()

x0             x1  ...           x830           x831
id                             ...                              
0.0  10.891876  832442.812375  ...   85338.558539  103088.664210
1.0  11.512994  832442.898114  ...   95695.020645  105161.109422
2.0  11.052185  832442.896307  ...   80253.299882  104177.051666
3.0  11.642076            NaN  ...            NaN  102746.516920
4.0  10.407121  832442.831424  ...  101815.745499  105163.749149

[5 rows x 832 columns]

## Imputing data

In [ ]:
# Method 1: imputing nan data and replace by median
#D_X_df = D_X_df.fillna(D_X_df.median())
#D_test_df = D_test_df.fillna(D_test_df.median())

# Method 2 (result same as 1): imputing nan data and replace by median
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy='median').fit(D_X_df)
D_X_df = pd.DataFrame(imp_mean.transform(D_X_df), index = D_X_df.index)
imp_mean = SimpleImputer(missing_values=np.nan, strategy='median').fit(D_test_df)
D_test_df = pd.DataFrame(imp_mean.transform(D_test_df), index = D_test_df.index)

# visualizing
D_X_df.head()
D_test_df.head(20)

# Method 3: advanced imputing
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer

# imp_mean = IterativeImputer(random_state=0).fit(D_X_df)
# D_X_df = pd.DataFrame(imp_mean.transform(D_X_df),index=D_X_df.index)

# imp_mean = IterativeImputer(random_state=0).fit(D_test_df)
# D_test_df = pd.DataFrame(imp_mean.transform(D_test_df),index=D_test_df.index)

# D_test_df.head(20)

0              1    ...            830            831
id                              ...                              
0.0    9.101943  832442.839400  ...   99818.695142  106668.615874
1.0   11.722077  832442.826314  ...  100761.264268  105023.965025
2.0   11.200277  832442.820359  ...  106440.456728  103405.273232
3.0    9.668873  832442.820901  ...   99818.695142  100531.960204
4.0   10.329962  832442.871842  ...  109797.625066  104849.648797
5.0    8.413879  832442.884113  ...   82662.485815  101539.952593
6.0    9.071789  832442.822903  ...  107564.170967  105122.219307
7.0   10.957082  832442.858015  ...   96880.592013  100453.162242
8.0   10.600499  832442.893470  ...  121654.728379  109301.962738
9.0    8.707614  832442.901854  ...   97516.729105  106489.145222
10.0   8.967393  832442.838199  ...  119071.038954  102661.454308
11.0  10.217481  832442.889530  ...   98783.153545  109676.921774
12.0  11.117558  832442.856471  ...   99893.302678  107264.439826
13.0   8.174608  832442.890680  ...  118373.995503  100578.239807
14.0   9.506562  832442.837356  ...   89193.770112  106820.946472
15.0  11.198049  832442.857946  ...  100167.934017  103140.984826
16.0  10.613045  832442.906858  ...   99818.695142  102023.940438
17.0  10.723322  832442.869458  ...  103767.815312  102188.989664
18.0  10.701383  832442.900612  ...   92122.927409  109196.084895
19.0   8.185341  832442.887075  ...  100175.763791  109897.510389

[20 rows x 832 columns]

## Converting to numpy array

In [ ]:
# defining numpy arrays 
D_X = np.array(D_X_df)
D_y = np.array(D_y_df)
D_test = np.array(D_test_df)

## Remove outliers in the training data set

In [ ]:
# Method 1: Isolation Forest
from sklearn.ensemble import IsolationForest

# Data dimensions before outlier removal
print(D_X.shape, D_y.shape)

# Identify outliers
iso = IsolationForest(contamination=0.08)
yhat = iso.fit_predict(D_X)

# select all rows that are not outliers
mask = yhat != -1
D_X, D_y = D_X[mask, :], D_y[mask]

# Dimensions after outlier removal
print(D_X.shape, D_y.shape)

(1212, 832) (1212, 1)
(1115, 832) (1115, 1)


In [ ]:
"""
# Method 2: Minimum Covariance Determinant (slow)
from sklearn.covariance import EllipticEnvelope

# Data dimensions before outlier removal
print(D_X.shape, D_y.shape)

ee = EllipticEnvelope(contamination=0.05)
yhat = ee.fit_predict(D_X)

# select all rows that are not outliers
mask = yhat != -1
D_X, D_y = D_X[mask, :], D_y[mask]

# Dimensions after outlier removal
print(D_X.shape, D_y.shape)
"""

(1115, 832) (1115, 1)


/usr/local/lib/python3.6/dist-packages/sklearn/covariance/_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


KeyboardInterrupt: ignored

# standardizing data - normalizing data around zero & unit std

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()

scaler.fit_transform(D_X)
D_X_stand = scaler.transform(D_X)
D_y = np.ravel(D_y)
D_test_stand = scaler.transform(D_test)

pd.DataFrame(D_test_stand).head()

0         1         2    ...       829       830       831
0 -0.991881 -0.736753  0.491744  ... -1.625340  0.002164  0.674352
1  1.826535 -1.219968 -0.559339  ...  0.528931  0.104419  0.047922
2  1.265248 -1.439871 -0.493102  ... -1.587615  0.720529 -0.568620
3 -0.382047 -1.419880  0.130784  ...  0.978188  0.002164 -1.663035
4  0.329071  0.461275 -1.502095  ...  1.772916  1.084733 -0.018473

[5 rows x 832 columns]

# Feature selection

In [ ]:
"""
from sklearn.feature_selection import SelectKBest, f_regression

print("Shape of D_X_stand before feature selection: {}".format(D_X_stand.shape))

selector = SelectKBest(score_func=f_regression, k=60)

selector.fit(D_X_stand, D_y)
selected_features = selector.get_support(indices=True)
unselected_features = [i for i in range(D_X_stand.shape[1]) if i not in selected_features]

# Remove features from training and test sets
X_features_removed = selector.transform(D_X_stand)
D_X_stand = X_features_removed
test_features_removed = np.delete(D_test_stand, unselected_features, axis=1)
D_test_stand = test_features_removed

print("Shape of D_X_stand after feature selection: {}".format(D_X_stand.shape))
"""

Shape of D_X_stand before feature selection: (1115, 832)
Shape of D_X_stand after feature selection: (1115, 60)


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


# K-fold

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(10,shuffle=True,random_state=13)

ind = np.arange(D_X.shape[0])

ind_splits = list(kf.split(ind))

## RBF regression with predict

In [ ]:
# getting time when starting training
import datetime
now = datetime.datetime.now()
unixTime = round(now.timestamp())

In [ ]:
"""
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

bestModel = None
score_best = float('-inf')
tempR2scores = []
tempModels = []

for idx, (train_index, val_index) in enumerate(ind_splits):

  D_X_train = D_X_stand[train_index,:]
  D_y_train = D_y[train_index]

  D_X_val = D_X_stand[val_index,:]
  D_y_val = D_y[val_index]

  #svr prediction
  svc = svm.SVR(kernel='rbf', verbose=True)
  model = svc.fit(D_X_train,D_y_train)
  score_m = model.score(D_X_val, D_y_val)

  y_pred = model.predict(D_X_val)

  # Rsquared score minimize
  # r2 = 0.5 + 0.5 * np.maximum(0, metrics.r2_score(D_y_val, y_pred))
  r2 = r2_score(D_y_val, y_pred)

  tempModels.append(model)
  tempR2scores.append(r2)
  
  # shouldn't choose lowest r2 score since this indicates overfitting, rather mean -> change this in the future
  if r2 > score_best:
    score_best = r2
    bestModel = model
    print(f'\nBest model has a score of {score_best}')
"""

[LibSVM]
Best model has a score of 0.432239613724405
[LibSVM]
Best model has a score of 0.49396209968123783
[LibSVM][LibSVM][LibSVM]
Best model has a score of 0.5812161803417897
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

bestModel = None
score_best = float('-inf')
tempR2scores = []
tempModels = []

for idx, (train_index, val_index) in enumerate(ind_splits):

  D_X_train = D_X_stand[train_index,:]
  D_y_train = D_y[train_index]

  D_X_val = D_X_stand[val_index,:]
  D_y_val = D_y[val_index]

  #Random Forest prediction
  rf = RandomForestRegressor(n_estimators = 1000)
  model = rf.fit(D_X_train,D_y_train)
  score_m = model.score(D_X_val, D_y_val)

  y_pred = model.predict(D_X_val)

  # Rsquared score minimize
  # r2 = 0.5 + 0.5 * np.maximum(0, metrics.r2_score(D_y_val, y_pred))
  r2 = r2_score(D_y_val, y_pred)

  tempModels.append(model)
  tempR2scores.append(r2)
  
  # shouldn't choose lowest r2 score since this indicates overfitting, rather mean -> change this in the future
  if r2 > score_best:
    score_best = r2
    bestModel = model
    print(f'\nBest model has a score of {score_best}')


Best model has a score of 0.5427632813407752

Best model has a score of 0.5882778550087304

Best model has a score of 0.6260499438092864


## predicting

In [ ]:
result_r = model.predict(D_test_stand)
result_r = np.transpose(np.array(result_r))

print(result_r.shape)

(776,)


## np arrays back to pandas dataframe TO DO: convert id to integer

In [ ]:
D_test_df.index
D_y_df.columns

df_r = pd.DataFrame(result_r,index=D_test_df.index)
df_out = pd.concat([df_r], axis=1, sort=False)
df_out.columns = D_y_df.columns
df_out.head()

y
id         
0.0  67.777
1.0  69.645
2.0  68.669
3.0  64.225
4.0  72.816

## saving dataframe TO DO: add time feature

In [ ]:
name = f'{baseDir}y_test_{unixTime}.csv'
print(f'File was saved under {name}')
df_out.to_csv(name, index=True, header = True, float_format='%.3f') #, compression='zip')

File was saved under /content/drive/My Drive/ETHZ/AML/Project_1/y_test_1602409046.csv
